In [1]:
import pandas as pd
from kiwipiepy import Kiwi
import re
from tqdm import tqdm

from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch
from torch.nn.functional import softmax

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 정리

In [2]:
# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

def read_data(path):
    survey = pd.read_csv(path, index_col=0)
    survey.fillna(0, inplace=True)
    survey = survey.reset_index()
    return survey

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

# 키위 전처리
def kiwi_clean(text):
    kiwi = Kiwi()
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
        else:
            kiwi_lem.append('')
    fin_txt = [txt for txt in kiwi_lem if txt != '']
    return ' '.join(fin_txt)

# 조사한 데이터 전체 정리
def get_data():
    # survey data 선언
    survey_data = read_data("./data/survey.csv")
    # content_id 중에서 댓글이 같이 추가된 데이터 정리하기
    survey_data['content_id'] = survey_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)
    # content_id 가 0인 데이터 제외하기 -> 추후 merge를 위해 content_id 모두 int type으로 변경
    survey_data = survey_data[survey_data['content_id'] != 0].reset_index().drop(columns=['index'])
    survey_data['content_id'] = survey_data['content_id'].astype(int)
    survey_data = survey_data[['content_id', 'guide', 'light', 'interior', 'story', 'probability', 'creativity', 'production', 'activity', 'scale', 'fear', 'device', 'fun', 'service']]
    # 전체 review data 불러오기
    review_all = pd.read_csv("./data/reviews.csv", index_col=0) 
    # 전체 review data 중에서 survey data에 있는 댓글만 가져오기
    survey_content = review_all.loc[review_all['id'].isin(survey_data['content_id'])][['id', 'content']]
    survey_content.columns = ['content_id', 'content']
    # survey, content merge 하기
    fin_data = pd.merge(survey_content, survey_data, on='content_id', how='left')
    fin_data = fin_data.drop_duplicates()
    return fin_data

def clean_kiwi_data():
    fin_df = get_data()
    fin_df['content'] = fin_df['content'].apply(cleaned_content)
    fin_df['content'] = fin_df['content'].apply(kiwi_clean)
    return fin_df

def clean_min_data():
    fin_df = get_data()
    fin_df['content'] = fin_df['content'].apply(cleaned_content)
    return fin_df

In [3]:
clear_data = clean_kiwi_data()
clear_data

,content_id,content,guide,light,interior,story,probability,creativity,production,activity,scale,fear,device,fun,service
0,513069,인 나쁘다 인테리어 문제 이다 각각 장치 작동 법 스스로 터득 되다 기본 어렵다 한...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,505540,칠포루바토 맞다 진짜 웃기다 신기 장치 가득가득 테,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,500819,괜히 칠바토 칠바토 그러다 아니다 방 탈 많이 하다 이거 평점 높다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,480104,인 오다 강남 흙 길 루바토 이다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,479813,이탬 루바토 이다 리뷰 보다 처음 장난 이다 하지만 옵저버 한 더 하다 이 태다 보...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,475313,방리 이 가다 무난하다 탈출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5019,461591,인 동생 은근 무섭다 기억,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5020,458794,조 어이없다,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5021,440808,기록 초기 방 탈출 테마 입문,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
clear_min_data = clean_min_data()
clear_min_data

,content_id,content,guide,light,interior,story,probability,creativity,production,activity,scale,fear,device,fun,service
0,513069,2인 / 나쁘지 않은 인테리어와 문제지만 각각의 장치들의 작동법을 스스로 터득해야 ...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,505540,칠포루바토 맞네! 진짜 웃기고 신기한 장치들이 가득가득한 테.,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,500819,괜히 칠바토 칠바토 그러는게 아니었네 나름 방탈 많이 했지만 이건 평점 높게 줄 만 하다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,480104,#344 4인 와 강남 흙길계의 루바토다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,479813,With 이탬 Gp루바토라는 리뷰를 보고 처음엔 장난이겠지 싶었다 하지만 옵저버로 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,475313,방린이들끼리 가서 무난하게 탈출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5019,461591,2인) 동생이랑. 은근 무서웠던거로 기억.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5020,458794,조나 어이없음 .,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5021,440808,"(기록용) 초기 방탈출테마, 입문용",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# train 코드 정리

In [3]:
# 데이터 확인 및 train, test 분리
def split_data(target):
    fin_data = pd.read_csv("./data/minbang_final.csv", index_col=0)
    final_target = fin_data[['content', target]]
    final_target = final_target.dropna()
    train_data = final_target.sample(frac=0.8, random_state=42)
    test_data = final_target.drop(train_data.index)
    return train_data, test_data

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

def resize_label(target):
    train_data, test_data = split_data(target)
    train_label = []
    for label in train_data[target].values:
        if label == -1:
            train_label.append([1., 0., 0.])
        elif label == 0:
            train_label.append([0., 1., 0.])
        elif label == 1:
            train_label.append([0., 0., 1.])

    test_label = []
    for label in test_data[target].values:
        if label == 0:
            test_label.append([0., 1., 0.])
        elif label == -1:
            test_label.append([1., 0., 0.])
        elif label == 1:
            test_label.append([0., 0., 1.])
    
    return train_label, test_label

def add_token(path):
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    # 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
    with open(path, "r") as add:
        add_data = add.read()
    addword = add_data.split('\n')
    # token에 새로운 단어 추가 
    tokenizer.add_tokens(addword)
    # token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
    # 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
    model.resize_token_embeddings(len(tokenizer))
    return tokenizer

def content_tokenize(target):
    train_data, test_data = split_data(target)
    train_label, test_label = resize_label(target)
    
    # train content 토큰화
    tokenized_train_sentences = tokenizer(
        list(train_data['content']),
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True
    )

    # test content 토큰화
    tokenized_test_sentences = tokenizer(
        list(test_data['content']),
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True
    )

    return tokenized_train_sentences, tokenized_test_sentences

def final_trainer(target, path, epoch, train_batch, eval_batch):
    train_data, test_data = split_data(target)
    train_label, test_label = resize_label(target)
    tokenized_train_sentences, tokenized_test_sentences = content_tokenize(target)
    # bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
    train_dataset = CurseDataset(tokenized_train_sentences, train_label)
    test_dataset = CurseDataset(tokenized_test_sentences, test_label)
    
    # hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
    training_args = TrainingArguments(
        output_dir = f'./model/{target}/{path}/chech_point',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
        num_train_epochs = epoch,              # 훈련 epoch 수
        per_device_train_batch_size = train_batch,    # 장치에 할당된 훈련 배치 크기
        per_device_eval_batch_size = eval_batch,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
        logging_dir = f'./model/{target}/{path}/logs',             # 훈련 중 로그 파일이 저장될 디렉토리
        logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
        save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
    )

    # hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    # train 진행
    trainer.train() 
    # train 완료 정보 저장
    trainer.save_model(f"./model/{target}/{path}/model")
    tokenizer.save_pretrained(f"./model/{target}/{path}/tokenizer")

In [4]:
# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = add_token("./data/add_data.txt")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
category_names = ['guide', 'light', 'interior', 'story', 'probability', 'creativity', 'production', 'activity', 'scale', 'fear', 'device', 'fun', 'service']

In [10]:
for category in tqdm(category_names):
    final_trainer(category, '2', 30, 16, 64)

  0%|          | 0/13 [00:00<?, ?it/s]

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_94299/3185649175.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                      
  0%|          | 0/13 [01:27<?, ?it/s]            

{'loss': 0.2149, 'learning_rate': 4.6679946879150064e-05, 'epoch': 1.99}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_94299/3185649175.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                      
  0%|          | 0/13 [02:54<?, ?it/s]             

{'loss': 0.1215, 'learning_rate': 4.335989375830013e-05, 'epoch': 3.98}


...

{'loss': 0.0154, 'learning_rate': 3.51925630810093e-06, 'epoch': 27.89}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_94299/3185649175.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
 92%|█████████▏| 12/13 [4:49:18<22:21, 1341.37s/it]

{'loss': 0.0136, 'learning_rate': 1.99203187250996e-07, 'epoch': 29.88}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_94299/3185649175.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
100%|██████████| 13/13 [4:49:24<00:00, 1335.75s/it]

{'train_runtime': 1343.6507, 'train_samples_per_second': 89.532, 'train_steps_per_second': 5.604, 'train_loss': 0.03685577344767759, 'epoch': 30.0}


# pred 코드 정리

In [6]:
clear_data = pd.read_csv('./data/minbang_final.csv')

In [7]:
def add_new_token(target, path):
    tokenizer = ElectraTokenizer.from_pretrained(f"./model/{target}/{path}/tokenizer")
    # 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
    with open("./data/add_data.txt", "r") as add:
        add_data = add.read()
    addword = add_data.split('\n')
    # token에 새로운 단어 추가 
    tokenizer.add_tokens(addword)
    # token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
    # 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
    model.resize_token_embeddings(len(tokenizer))
    return tokenizer

def read_saved_model(target, path):
    # 저장된 모델과 토크나이저를 불러오기
    model_path = f"./model/{target}/{path}/model"
    model = ElectraForSequenceClassification.from_pretrained(model_path)
    tokenizer = add_new_token(target, path)
    
    return model, tokenizer

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

# 함수 정의
def classify_clear_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities
    
def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

def pred_data(target, path):
    train_data, test_data = split_data(target)
    model, tokenizer = read_saved_model(target, path)
    model.resize_token_embeddings(len(tokenizer))
    # print(test_data)
    
    pred_list = []
    for sent in test_data['content']:
        # print(sent)
        pred, _ = classify_clear_text(sent)
        pred_list.append(pred)

    test_data['pred'] = pred_list
    test_data['pred'] = test_data['pred'].apply(change_num)
    print(f'{target} 0포함 예측률', '\n', classification_report(test_data[target], test_data['pred']))
    
    test_data2 = test_data[test_data[target] != 0]
    print(f'{target} 0제외 예측률', '\n', classification_report(test_data2[target], test_data2['pred']))
    return test_data

In [9]:
category_names = ['guide', 'light', 'interior', 'story', 'probability', 'creativity', 'production', 'activity', 'scale', 'fear', 'device', 'fun', 'service']
for category in tqdm(category_names):
    pred_data(category, '1')

  0%|          | 0/13 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

guide 0포함 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        31
         0.0       0.96      0.99      0.98       963
         1.0       0.00      0.00      0.00         9

    accuracy                           0.95      1003
   macro avg       0.32      0.33      0.33      1003
weighted avg       0.92      0.95      0.94      1003

guide 0제외 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      31.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00       9.0

    accuracy                           0.00      40.0
   macro avg       0.00      0.00      0.00      40.0
weighted avg       0.00      0.00      0.00      40.0



/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

light 0포함 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        14
         0.0       0.98      0.99      0.99       988
         1.0       0.00      0.00      0.00         1

    accuracy                           0.98      1003
   macro avg       0.33      0.33      0.33      1003
weighted avg       0.97      0.98      0.97      1003

light 0제외 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      14.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00       1.0

    accuracy                           0.00      15.0
   macro avg       0.00      0.00      0.00      15.0
weighted avg       0.00      0.00      0.00      15.0



/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

...

device 0포함 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        72
         0.0       0.89      0.99      0.94       889
         1.0       0.00      0.00      0.00        42

    accuracy                           0.88      1003
   macro avg       0.30      0.33      0.31      1003
weighted avg       0.78      0.88      0.83      1003

device 0제외 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      72.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00      42.0

    accuracy                           0.00     114.0
   macro avg       0.00      0.00      0.00     114.0
weighted avg       0.00      0.00      0.00     114.0



/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

fun 0포함 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        13
         0.0       0.80      0.99      0.88       800
         1.0       0.00      0.00      0.00       190

    accuracy                           0.79      1003
   macro avg       0.27      0.33      0.29      1003
weighted avg       0.64      0.79      0.71      1003

fun 0제외 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      13.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00     190.0

    accuracy                           0.00     203.0
   macro avg       0.00      0.00      0.00     203.0
weighted avg       0.00      0.00      0.00     203.0



/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/Bang/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

service 0포함 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         9
         0.0       0.98      0.99      0.99       982
         1.0       0.00      0.00      0.00        12

    accuracy                           0.97      1003
   macro avg       0.33      0.33      0.33      1003
weighted avg       0.96      0.97      0.97      1003

service 0제외 예측률 
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       9.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00      12.0

    accuracy                           0.00      21.0
   macro avg       0.00      0.00      0.00      21.0
weighted avg       0.00      0.00      0.00      21.0

